In [59]:
import pandas as pd
import matplotlib

In [51]:
vdh_foodPermits = pd.read_csv('VDH_FoodPermits.csv')

vdh_foodPermits

,HealthDistrict,establishmentName,permitName,permitNumber,program,PermitType,status,RiskCategory,catering,commissary,...,inspectionFrequency,ApplicationDate,OriginalIssueDate,PermitIssueDate,ExpirationDate,FacilityAddress,city,zip,FIPSCode,LocalityName
0,EHD - Rappahannock,NaN,A-La-Carte by Weddings by Ginny,JCRS-5W9Q8L,Food Establishment,Restaurant/Caterer,Surrendered,1,NaN,NaN,...,NaN,2/12/2004,6/18/2004,6/18/2004,6/30/2005,NaN,NaN,NaN,NaN,NaN
1,EHD - Prince William,NaN,Chiyoshi,AJOS-ARKQPG,Food Establishment,Carry Out Food Service Only,Business Closed,3,NO,NO,...,NaN,9/22/2017,3/29/2018,4/1/2019,3/31/2022,NaN,NaN,NaN,NaN,NaN
2,EHD - Three Rivers,NaN,Monroe Bay Landing,JKAZ-59KLBD,Food Establishment,Restaurant,Surrendered,4,NaN,NaN,...,NaN,1/3/1994,2/10/1995,2/13/2003,2/28/2004,NaN,NaN,NaN,NaN,NaN
3,EHD - New River,NaN,Narrows High School-girls Basketball,ACOS-5NUP2H,Food Establishment,Temporary restaurant,Surrendered,NaN,NaN,NaN,...,NaN,6/25/2003,6/25/2003,7/4/2003,7/4/2003,NaN,NaN,NaN,NaN,NaN
4,EHD - Virginia Beach,NaN,No. One Chinese Restaurant,JLLD-5CLR4E,Food Establishment,Full Service Restaurant,Surrendered,4,NaN,NaN,...,NaN,7/1/2003,7/8/2003,7/1/2008,7/31/2009,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,EHD - Prince William,Wendy’s #6589,Wendy's #6589,#REF!,Food Establishment,NaN,Permitted,2,NO,NO,...,NaN,8/18/2021,10/6/2021,10/6/2021,10/31/2023,2410 Prince William Pkwy,Woodbridge,22192,NaN,NaN
8042,EHD - Prince William,Wendy’s #6589,Wendy's #139,JSUZ-55HQY2,Food Establishment,Fast Food Restaurant,Surrendered,2,NaN,NaN,...,6.0,9/10/2002,11/19/2002,12/1/2016,11/30/2017,2410 Prince William Pkwy,Woodbridge,22192,NaN,NaN
8043,EHD - Prince William,Wendy’s #6589,Wendy's #6589,AJOS-AMUM8G,Food Establishment,Fast Food Restaurant,Business Closed,NaN,NO,NO,...,NaN,5/17/2017,8/3/2017,6/1/2019,5/31/2021,2410 Prince William Pkwy,Woodbridge,22192,NaN,NaN
8044,EHD - Alleghany-Roanoke,3096 Colonial Avenue SW,Subway #63400 @ Virginia Western Community Col...,#REF!,Food Establishment,Fast Food Restaurant,Business Closed,1,NO,NO,...,6.0,7/25/2014,8/19/2014,9/1/2019,8/31/2020,3096 Colonial Avenue SW,Roanoke,24015,NaN,NaN


### Cleaning Dataset

In [52]:
# Considering the first 300 rows
clean_vdh = vdh_foodPermits.loc[0:300, 'HealthDistrict': 'RiskCategory']

# Switch object type to factor
clean_vdh['HealthDistrict'] = clean_vdh['HealthDistrict'].astype('string')

# replace NaN values from Establishment Name into thier permit Name for analysis purpose
clean_vdh['establishmentName'] = clean_vdh['establishmentName'].astype('string')
clean_vdh['establishmentName'] = clean_vdh['establishmentName'].fillna(clean_vdh['permitName'])

# Removed EHD from Health Districts for easy access of location/Districts
clean_vdh.HealthDistrict = clean_vdh.HealthDistrict.map(lambda b: b.replace("EHD - ", ""))

# Remove columns that are insignificant
clean_vdh = clean_vdh.drop(columns=["permitNumber", "program", "status", "RiskCategory"])
clean_vdh

,HealthDistrict,establishmentName,permitName,PermitType
0,Rappahannock,A-La-Carte by Weddings by Ginny,A-La-Carte by Weddings by Ginny,Restaurant/Caterer
1,Prince William,Chiyoshi,Chiyoshi,Carry Out Food Service Only
2,Three Rivers,Monroe Bay Landing,Monroe Bay Landing,Restaurant
3,New River,Narrows High School-girls Basketball,Narrows High School-girls Basketball,Temporary restaurant
4,Virginia Beach,No. One Chinese Restaurant,No. One Chinese Restaurant,Full Service Restaurant
...,...,...,...,...
296,Western Tidewater,Franklin Church of God - 2012 Summer Food Serv...,Southampton County High School,Summer Food Service Program Feeding Site
297,Western Tidewater,Franklin Church of God - 2012 Summer Food Serv...,Franklin Church of God,Summer Food Service Program Kitchen
298,Richmond,Soul Bistro Home of the Heavy Plates,Soul Bistro Home of the Heavy Plates,Fast Food Restaurant
299,Richmond,Tropicana Restaurant & Catering,Tropicana Restaurant & Catering,Full Service Restaurant/Caterer


### API and Mapping health district to county level

In [22]:
# First step: Obtain an API key from the US Census Bureau's website
api_key = 'af7d72e7b5f0ccde2289b15f56500e899c752023'

In [23]:
# Second step: Install the census and us Python packages

In [24]:
# Third step: Import the necessary packages in this Python script
from census import Census
from us import states

In [25]:
# Fourth step: Set up API key by creating an instance of the Census class and passing in API key as a parameter
c = Census(api_key)

In [26]:
# Final steps: Obtain a list of the counties in Virginia from the Census Bureau API
# Use method to get a list of all available tables in the API
counties = c.acs5.state_county_tract("NAME", state_fips = states.VA.fips, county_fips = "*", tract="*")

In [27]:
# The above will return a list of dictionaries, where each dictionary corresponds 
# to a county and contains the county's name and FIPS code.

# Federal Information Processing System
# FIPS code: FIPS codes are numbers which uniquely identify geographic areas.

In [28]:
# Map the health districts to corresponding counties
# To do this, you can create a dictionary that maps the FIPS codes to their corresponding counties, and 
# then use this dictionary to match each health district to its corresponding county.

county_dict = {}
for county in counties:
    county_dict[county['county']] = county['NAME']

# This creates a dictionary where the keys are the county FIPS codes and the values are the county names.

In [29]:
# Add a new column to your dataset that contains the county name for each health district. 

In [46]:
# Create a new column to store the corresponding county for each health district


In [54]:
# Create a new column that contains the county name for each restaurant permit
#clean_vdh['county'] = clean_vdh['health_district'].map(district_county_table)

### Cleaning second Dataset and Merging

In [56]:
locality = pd.read_csv('Locality-to-HD-to-HPR.csv')
locality = locality.rename(columns={'Unnamed: 0': 'CityCounty', 'Unnamed: 1': 'CountyFIPS','Unnamed: 2': 'HealthDistrict'})
locality = locality.drop(columns=["Unnamed: 3"])
locality = locality.drop([0,1])
locality

,CityCounty,CountyFIPS,HealthDistrict
2,Accomack,51001,Eastern Shore
3,Albemarle,51003,Thomas Jefferson
4,Alexandria,51510,Alexandria
5,Alleghany,51005,Alleghany
6,Amelia,51007,Piedmont
...,...,...,...
132,Williamsburg,51830,Peninsula
133,Winchester,51840,Lord Fairfax
134,Wise,51195,Lenowisco
135,Wythe,51197,Mount Rogers


In [62]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


merged = pd.merge(clean_vdh, locality, on="HealthDistrict")
merged = merged.drop_duplicates(subset=["establishmentName", "permitName"])
merged

,HealthDistrict,establishmentName,permitName,PermitType,CityCounty,CountyFIPS
0,Rappahannock,A-La-Carte by Weddings by Ginny,A-La-Carte by Weddings by Ginny,Restaurant/Caterer,Caroline,51033
10,Rappahannock,"Gabby's Seafood, LLC","Gabby's Seafood, LLC",Mobile Food Unit,Caroline,51033
15,Rappahannock,Grioli’s Italian Bistro,Grioli's Italian Bistro,Full Service Restaurant,Caroline,51033
20,Rappahannock,Ponderosa,Ponderosa,Full Service Restaurant,Caroline,51033
30,Rappahannock,Virginia Barbeque Company,Virginia Barbeque Company,Full Service Restaurant,Caroline,51033
35,Rappahannock,Speedy’s Mart (Ladysmith),Speedy's Mart (Ladysmith),Restaurant,Caroline,51033
40,Rappahannock,Bernice Varieties,Bernice Varieties,Caterer,Caroline,51033
50,Rappahannock,Dinty Moore’s,Dinty Moore's,Full Service Restaurant,Caroline,51033
55,Rappahannock,"Granny’s ""Home Cookin""","Granny's ""Home Cookin""",Full Service Restaurant,Caroline,51033
60,Rappahannock,"John’s Place, LLC.",John's Place,Full Service Restaurant,Caroline,51033
